In [2]:
# Core
import numpy as np
import pandas as pd
import os
import json

# Dataset prepration
#from transformers import TFAutoModel, AutoModel
from transformers import AutoTokenizer
from datasets import Dataset
#from torch.utils.data import DataLoader

# Model, hyperparameter search, evaluation
import torch
import optuna
from transformers import BertPreTrainedModel, TrainingArguments
from transformers.models.roberta.modeling_roberta import (
    RobertaClassificationHead,
    RobertaConfig,
    RobertaModel,
)
from torch.nn import CrossEntropyLoss
import evaluate

from transformers import get_scheduler, Trainer
from torch.optim import AdamW
from tqdm.auto import tqdm

# Define functions

These code blocks contain functions that are used multiple times, by each section.

In [3]:
# Prepare variables for indexing
classes = ["Explicit", "Implicit", "Non-abusive"]
labels = ["E1", "E2", "E3", "I1", "I2", "I3", "I4", "I5", "I6", "I7"]

### `load_csv`
Loads a dataset split from a constant directory.
To change the directory, modify the loaded CSV at line $10$.

- `dataset: str (train, test, split)` — Split to load.
- `use_stopwords: bool` — Whether to use a dataset with stopwords or not.

Returns: `DataFrame` containing the loaded CSV.

In [4]:
def load_csv(dataset, use_stopwords):
    if dataset not in ["train", "validate", "test"]:
        raise Exception("Invalid split.")
    if type(use_stopwords) != bool:
        raise Exception("Stop words must be specified in boolean.")
    
    stopwords = "With Stopwords"
    if not use_stopwords:
        stopwords = "Without Stopwords"
    df = pd.read_csv(f"./Data/{stopwords}/{dataset}.csv")
    
    return df

### `format_dataset`
Formats a DataFrame for HuggingFace use.

- `df: DataFrame` — DataFrame to convert.
- `cv_indices: list` — Indices to create stratified CV datasets.

Returns: `list`, `list` containing HuggingFace `Datasets` for training and testing.

In [5]:
def format_dataset(df):
    df.rename({"Text": "text", "Class": "labels"}, axis=1, inplace=True)
    df.drop(labels, axis=1, inplace=True)
    
    dataset = Dataset.from_pandas(df)
    
    # Convert pd labels to huggingface ClassLabels for stratifying
    dataset = dataset.class_encode_column("labels")
    
    dataset = dataset.map(tokenize_function, batched=True)
    
    # Convert datasets to pytorch format
    dataset = dataset.remove_columns(["text"])
    dataset.set_format("torch")
        
    return dataset

### `save_preds`
Saves the predictions of a model.

- `filename: str` — Filename to use. Must include the `.json` extension.
- `data: list` — Contains the data to save.

In [6]:
def save_preds(filename, data):
    # Serializing json
    json_object = json.dumps({"predictions": data}, indent=4)

    # Writing to sample.json
    with open(filename, "w") as outfile:
        outfile.write(json_object)

# Standard multiclass classifier RoBERTa

## Preparation for models

Prepares the RoBERTa model's classification head based on a repository by [Chanda](https://pchanda.github.io/Roberta-FineTuning-for-Classification/).

In addition prepares the `compute_metrics` function to be used by the HuggingFace `Trainer` class.

In [7]:
# Prepare classification head for pretrained RoBERTa
class RobertaAbusiveClassification(BertPreTrainedModel):
    
    def __init__(self, config):
        super(RobertaAbusiveClassification, self).__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config)
        self.classifier = RobertaClassificationHead(config)
        
    def forward(self, input_ids, attention_mask, labels):
        outputs = self.roberta(input_ids,attention_mask=attention_mask)
        sequence_output = outputs[0]
        logits = self.classifier(sequence_output)

        outputs = (logits,) + outputs[2:]
        
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [7]:
def compute_metrics(eval_pred):
    global predictions
    acc_metric = evaluate.load("accuracy")
    pre_metric = evaluate.load("precision")
    rec_metric = evaluate.load("recall")
    f1_metric = evaluate.load("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = acc_metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = pre_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = rec_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    
    # Calculate per-class metrics
    results = []
    for i in range(0, 3):
        x = labels[predictions == i]
        y = predictions[predictions == i]
        
        class_accuracy = acc_metric.compute(predictions=y, references=x)["accuracy"]
        class_precision = pre_metric.compute(predictions=y, references=x, average="weighted")["precision"]
        class_recall = rec_metric.compute(predictions=y, references=x, average="weighted")["recall"]
        class_f1 = f1_metric.compute(predictions=y, references=x, average="weighted")["f1"]
        results += [class_accuracy, class_precision, class_recall, class_f1]
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1":f1,
           "NA_accuracy": results[0], "NA_precision": results[1], "NA_recall": results[2], "NA_f1": results[3],
           "E_accuracy": results[4], "E_precision": results[5], "E_recall": results[6], "E_f1": results[7],
           "I_accuracy": results[8], "I_precision": results[9], "I_recall": results[10], "I_f1": results[11]}

In [8]:
# Variables
num_labels = 3
load_dir = "jcblaise/roberta-tagalog-base"

# Get tokenizer from repository
tokenizer = AutoTokenizer.from_pretrained(load_dir, model_max_length=256)

def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation=True)

In [20]:
# Load and format CSVs
train_df = load_csv("train", True)
train_dataset = format_dataset(train_df)

test_df = load_csv("test", True)
test_dataset = format_dataset(test_df)

val_df = load_csv("validate", True)
val_dataset = format_dataset(val_df)

## Train initial model

In [10]:
save_dir = "./Models/standard-initial"

# Create directories
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [11]:
# Attach classification head and prepare model for trainer
config = RobertaConfig.from_pretrained(load_dir, num_labels=num_labels)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_args = TrainingArguments(
    output_dir = save_dir,
    report_to = "none")

# Initialize model
model = RobertaAbusiveClassification.from_pretrained(load_dir, config=config)
model.to(device)
model.train()

# Initialize Trainer class and train
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics)
result = trainer.train()

# Evaluate model and save metrics and predictions
model.eval()
predictions = []
metrics = trainer.evaluate()
trainer.save_metrics('validation', metrics)
save_preds(f'{save_dir}/validation_predictions.json', predictions.tolist())

# Save trained model
model.save_pretrained(f'{save_dir}/model')
tokenizer.save_pretrained(f'{save_dir}/model')

Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAbusiveClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weig

Step,Training Loss


C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('./Models/standard-initial/model\\tokenizer_config.json',
 './Models/standard-initial/model\\special_tokens_map.json',
 './Models/standard-initial/model\\vocab.json',
 './Models/standard-initial/model\\merges.txt',
 './Models/standard-initial/model\\added_tokens.json',
 './Models/standard-initial/model\\tokenizer.json')

In [12]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
val_df = pd.DataFrame(values.reshape(4, 4), columns=colnames, index=["Average"] + classes)

print("Validation set metrics:")
val_df

Validation set metrics:


,Accuracy,Precision,Recall,F1
Average,0.712389,0.717259,0.712389,0.713154
Explicit,0.757576,0.573921,0.757576,0.653083
Implicit,0.760000,0.577600,0.760000,0.656364
Non-abusive,0.635294,0.403599,0.635294,0.493610


In [13]:
predictions = []
metrics = trainer.evaluate(test_dataset)
trainer.save_metrics('test', metrics)
save_preds(f'{save_dir}/test_predictions.json', predictions.tolist())

C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
test_df = pd.DataFrame(values.reshape(4, 4), columns=colnames, index=["Average"] + classes)

print("Test set metrics:")
test_df

Test set metrics:


,Accuracy,Precision,Recall,F1
Average,0.682819,0.694285,0.682819,0.686960
Explicit,0.788732,0.622099,0.788732,0.695575
Implicit,0.757143,0.573265,0.757143,0.652497
Non-abusive,0.534884,0.286101,0.534884,0.372798


## Fine-tune pre-trained model

Fine tuning using [Optuna with transformers](https://python-bloggers.com/2022/08/hyperparameter-tuning-a-transformer-with-optuna/)

In [15]:
# Variables
save_dir = "./Models/standard-tuning"

In [19]:
def objective(trial: optuna.Trial):
    config = RobertaConfig.from_pretrained(load_dir, num_labels=num_labels)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = RobertaAbusiveClassification.from_pretrained(load_dir, config=config)
    model.to(device)
    model.train()
    
    training_args = TrainingArguments(         
        output_dir= save_dir, 
        # Test for different optimizer
        # optimizer = trial.suggest_categorical("optimizer", ["MomentumSGD", "Adam"])
        optim = "adamw_torch",
        learning_rate = trial.suggest_float("learning_rate", 4e-5, 0.01, log=True),         
        weight_decay = trial.suggest_float("weight_decay", 4e-5, 0.01, log=True),         
        num_train_epochs = trial.suggest_int("num_train_epochs", low=2, high=5),         
        per_device_train_batch_size = 8,         
        per_device_eval_batch_size = 8,
        report_to = "none")

    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = train_dataset,
        eval_dataset = val_dataset)  
    
    result = trainer.train()     
    return result.training_loss

In [20]:
study = optuna.create_study(study_name="hp-search-roberta", direction="minimize") 
study.optimize(func=objective, n_trials=128)

[I 2023-07-18 13:21:23,456] A new study created in memory with name: hp-search-roberta
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAbusiveClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.dense.

Step,Training Loss


[I 2023-07-18 13:22:47,105] Trial 0 finished with value: 1.3856196582765508 and parameters: {'learning_rate': 0.003649141609739691, 'weight_decay': 0.00040036096382604523, 'num_train_epochs': 3}. Best is trial 0 with value: 1.3856196582765508.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights o

Step,Training Loss


[I 2023-07-18 13:24:13,876] Trial 1 finished with value: 1.1972683784656954 and parameters: {'learning_rate': 0.0009099928570682495, 'weight_decay': 0.003792662235843005, 'num_train_epochs': 3}. Best is trial 1 with value: 1.1972683784656954.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of

...

[I 2023-07-18 18:09:15,479] Trial 125 finished with value: 0.3342960759213096 and parameters: {'learning_rate': 7.349246105317668e-05, 'weight_decay': 0.0002515244149617616, 'num_train_epochs': 5}. Best is trial 115 with value: 0.3184867629431244.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weigh

Step,Training Loss
500,0.449100


[I 2023-07-18 18:11:39,039] Trial 126 finished with value: 0.34231528339529393 and parameters: {'learning_rate': 5.905293791513107e-05, 'weight_decay': 0.00020805955756182423, 'num_train_epochs': 5}. Best is trial 115 with value: 0.3184867629431244.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some wei

Step,Training Loss
500,0.435700


[I 2023-07-18 18:14:04,845] Trial 127 finished with value: 0.33041852631963287 and parameters: {'learning_rate': 8.518707989810698e-05, 'weight_decay': 0.0003751827805596012, 'num_train_epochs': 5}. Best is trial 115 with value: 0.3184867629431244.


In [21]:
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])

print("Optimal hyperparamers:")
print(f'Learning rate\t: {best_lr}')
print(f'Weight decay\t: {best_weight_decay}')
print(f'Epochs\t\t: {best_epoch}')

Optimal hyperparamers:
Learning rate	: 7.456002813629831e-05
Weight decay	: 0.0002953648171116107
Epochs		: 5


In [22]:
# Save best hyperparameters
data = {
    'learning_rate': best_lr,
    'weight_decay': best_weight_decay,
    'epoch': best_epoch
}

# Serializing json
json_object = json.dumps(data, indent=4)

# Writing to sample.json
with open(f'{save_dir}/best_hyperparameters.json', "w") as outfile:
    outfile.write(json_object)

## Training the best model

In [23]:
# Variables
load_dir = "jcblaise/roberta-tagalog-base"
save_dir = "./Models/standard-final"
hyperparameter_dir = "./Models/standard-tuning"

In [24]:
# Load saved hyperparameters
with open(f'{hyperparameter_dir}/best_hyperparameters.json') as f:
    hyperparameters = json.load(f)

hyperparameters

{'learning_rate': 7.456002813629831e-05,
 'weight_decay': 0.0002953648171116107,
 'epoch': 5}

In [25]:
config = RobertaConfig.from_pretrained(load_dir, num_labels=num_labels)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = RobertaAbusiveClassification.from_pretrained(load_dir, config=config)
model.to(device)
model.train()

training_args = TrainingArguments(
    output_dir= save_dir,
    learning_rate=hyperparameters['learning_rate'],
    weight_decay=hyperparameters['weight_decay'],
    num_train_epochs=hyperparameters['epoch'],
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    report_to = "none")

# Initialize Trainer class and train
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics)
result = trainer.train()

# Evaluate model and save metrics and predictions
model.eval()
predictions = []
metrics = trainer.evaluate()
trainer.save_metrics('validation', metrics)
save_preds(f'{save_dir}/validation_predictions.json', predictions.tolist())

# Save trained model
model.save_pretrained(f'{save_dir}/model')
tokenizer.save_pretrained(f'{save_dir}/model')

Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAbusiveClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weig

Step,Training Loss
500,0.428300


C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('./Models/standard-final/model\\tokenizer_config.json',
 './Models/standard-final/model\\special_tokens_map.json',
 './Models/standard-final/model\\vocab.json',
 './Models/standard-final/model\\merges.txt',
 './Models/standard-final/model\\added_tokens.json',
 './Models/standard-final/model\\tokenizer.json')

In [26]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
val_df = pd.DataFrame(values.reshape(4, 4), columns=colnames, index=["Average"] + classes)

print("Validation set metrics:")
val_df

Validation set metrics:


,Accuracy,Precision,Recall,F1
Average,0.676991,0.688843,0.676991,0.678448
Explicit,0.750000,0.562500,0.750000,0.642857
Implicit,0.746667,0.557511,0.746667,0.638372
Non-abusive,0.571429,0.326531,0.571429,0.415584


In [27]:
predictions = []
metrics = trainer.evaluate(test_dataset)
trainer.save_metrics('test', metrics)
save_preds(f'{save_dir}/test_predictions.json', predictions.tolist())

C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
test_df = pd.DataFrame(values.reshape(4, 4), columns=colnames, index=["Average"] + classes)

print("Test set metrics:")
test_df

Test set metrics:


,Accuracy,Precision,Recall,F1
Average,0.718062,0.730146,0.718062,0.721571
Explicit,0.830769,0.690178,0.830769,0.753975
Implicit,0.776316,0.602666,0.776316,0.678558
Non-abusive,0.581395,0.338021,0.581395,0.427497


# Hierarchical multiclass classifier RoBERTa

## Preparation for models

In [8]:
def compute_metrics_main(eval_pred):
    global predictions
    acc_metric = evaluate.load("accuracy")
    pre_metric = evaluate.load("precision")
    rec_metric = evaluate.load("recall")
    f1_metric = evaluate.load("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = acc_metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = pre_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = rec_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    
    # Calculate per-class metrics
    results = []
    for i in range(0, 2):
        x = labels[predictions == i]
        y = predictions[predictions == i]
        
        class_accuracy = acc_metric.compute(predictions=y, references=x)["accuracy"]
        class_precision = pre_metric.compute(predictions=y, references=x, average="weighted")["precision"]
        class_recall = rec_metric.compute(predictions=y, references=x, average="weighted")["recall"]
        class_f1 = f1_metric.compute(predictions=y, references=x, average="weighted")["f1"]
        results += [class_accuracy, class_precision, class_recall, class_f1]
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1":f1,
           "NA_accuracy": results[0], "NA_precision": results[1], "NA_recall": results[2], "NA_f1": results[3],
           "A_accuracy": results[4], "A_precision": results[5], "A_recall": results[6], "A_f1": results[7]}

In [9]:
def compute_metrics_sub(eval_pred):
    global predictions
    acc_metric = evaluate.load("accuracy")
    pre_metric = evaluate.load("precision")
    rec_metric = evaluate.load("recall")
    f1_metric = evaluate.load("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = acc_metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = pre_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = rec_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    
    # Calculate per-class metrics
    results = []
    for i in range(0, 2):
        x = labels[predictions == i]
        y = predictions[predictions == i]
        
        class_accuracy = acc_metric.compute(predictions=y, references=x)["accuracy"]
        class_precision = pre_metric.compute(predictions=y, references=x, average="weighted")["precision"]
        class_recall = rec_metric.compute(predictions=y, references=x, average="weighted")["recall"]
        class_f1 = f1_metric.compute(predictions=y, references=x, average="weighted")["f1"]
        results += [class_accuracy, class_precision, class_recall, class_f1]
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1":f1,
           "E_accuracy": results[0], "E_precision": results[1], "E_recall": results[2], "E_f1": results[3],
           "I_accuracy": results[4], "I_precision": results[5], "I_recall": results[6], "I_f1": results[7]}

In [10]:
def load_preds(directory):
    # Load files
    f = open(f'{directory}/test_predictions.json')
    preds = [json.load(f)]
    f.close()

    return np.array(preds[0]['predictions'])

In [11]:
# Variables
num_labels = 2
load_dir = "jcblaise/roberta-tagalog-base"

# Get tokenizer from repository
tokenizer = AutoTokenizer.from_pretrained(load_dir, model_max_length=256)

def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation=True)

In [13]:
def convert_abusive_encoding(x):
    if x != 0:
        return 1
    return 0

# Load and format CSVs
df = load_csv("train", True)
df["Class"] = df["Class"].apply(convert_abusive_encoding)
main_train_dataset = format_dataset(df)

df = load_csv("test", True)
df["Class"] = df["Class"].apply(convert_abusive_encoding)
main_test_dataset = format_dataset(df)

df = load_csv("validate", True)
df["Class"] = df["Class"].apply(convert_abusive_encoding)
main_val_dataset = format_dataset(df)

In [14]:
def convert_sub_encoding(x):
    if x == 2:
        return 1
    return 0

# Prepare datasets composed of abusive text
df = load_csv("train", True)
df = df[df["Class"] != 0]
sub_train_dataset = format_dataset(df)

# Prepare datasets composed of abusive text
df = load_csv("test", True)
df = df[df["Class"] != 0]
sub_test_dataset = format_dataset(df)

df = load_csv("validate", True)
df = df[df["Class"] != 0]
sub_val_dataset = format_dataset(df)

## Train initial models

### Main model (NA v A)

In [34]:
save_dir = "./Models/hierarchical-initial/main"

# Create directories
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [35]:
# Attach classification head and prepare model for trainer
config = RobertaConfig.from_pretrained(load_dir, num_labels=num_labels)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_args = TrainingArguments(
    output_dir = save_dir,
    report_to = "none")

# Initialize model
model = RobertaAbusiveClassification.from_pretrained(load_dir, config=config)
model.to(device)
model.train()

# Initialize Trainer class and train
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = main_train_dataset,
    eval_dataset = main_val_dataset,
    compute_metrics = compute_metrics_main)
result = trainer.train()

# Evaluate model and save metrics and predictions
model.eval()
predictions = []
metrics = trainer.evaluate()
trainer.save_metrics('validation', metrics)
save_preds(f'{save_dir}/validation_predictions.json', predictions.tolist())

# Save trained model
model.save_pretrained(f'{save_dir}/model')
tokenizer.save_pretrained(f'{save_dir}/model')

Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAbusiveClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weig

Step,Training Loss


C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('./Models/hierarchical-initial/main/model\\tokenizer_config.json',
 './Models/hierarchical-initial/main/model\\special_tokens_map.json',
 './Models/hierarchical-initial/main/model\\vocab.json',
 './Models/hierarchical-initial/main/model\\merges.txt',
 './Models/hierarchical-initial/main/model\\added_tokens.json',
 './Models/hierarchical-initial/main/model\\tokenizer.json')

In [36]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
val_df = pd.DataFrame(values.reshape(3, 4), columns=colnames, index=["Average", "Non-Abusive", "Abusive"])

print("Validation set metrics:")
val_df

Validation set metrics:


,Accuracy,Precision,Recall,F1
Average,0.814159,0.810454,0.814159,0.808711
Non-Abusive,0.770492,0.593658,0.770492,0.670613
Abusive,0.830303,0.689403,0.830303,0.753321


In [37]:
predictions = []
metrics = trainer.evaluate(main_test_dataset)
trainer.save_metrics('test', metrics)
save_preds(f'{save_dir}/test_predictions.json', predictions.tolist())

C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
test_df = pd.DataFrame(values.reshape(3, 4), columns=colnames, index=["Average", "Non-Abusive", "Abusive"])

print("Test set metrics:")
test_df

Test set metrics:


,Accuracy,Precision,Recall,F1
Average,0.837004,0.835006,0.837004,0.835560
Non-Abusive,0.774648,0.600079,0.774648,0.676280
Abusive,0.865385,0.748891,0.865385,0.802934


### Sub model (E v I)

In [15]:
save_dir = "./Models/hierarchical-initial/sub"

# Create directories
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [41]:
# Attach classification head and prepare model for trainer
config = RobertaConfig.from_pretrained(load_dir, num_labels=num_labels)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_args = TrainingArguments(
    output_dir = save_dir,
    report_to = "none")

# Initialize model
model = RobertaAbusiveClassification.from_pretrained(load_dir, config=config)
model.to(device)
model.train()

# Initialize Trainer class and train
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = sub_train_dataset,
    eval_dataset = sub_val_dataset,
    compute_metrics = compute_metrics_sub)
result = trainer.train()

# Evaluate model and save metrics and predictions
model.eval()
predictions = []
metrics = trainer.evaluate()
trainer.save_metrics('validation', metrics)
save_preds(f'{save_dir}/validation_predictions.json', predictions.tolist())

# Save trained model
model.save_pretrained(f'{save_dir}/model')
tokenizer.save_pretrained(f'{save_dir}/model')

Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAbusiveClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weig

Step,Training Loss


C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('./Models/hierarchical-initial/sub/model\\tokenizer_config.json',
 './Models/hierarchical-initial/sub/model\\special_tokens_map.json',
 './Models/hierarchical-initial/sub/model\\vocab.json',
 './Models/hierarchical-initial/sub/model\\merges.txt',
 './Models/hierarchical-initial/sub/model\\added_tokens.json',
 './Models/hierarchical-initial/sub/model\\tokenizer.json')

In [42]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
val_df = pd.DataFrame(values.reshape(3, 4), columns=colnames, index=["Average", "Explicit", "Implicit"])

print("Validation set metrics:")
val_df

Validation set metrics:


,Accuracy,Precision,Recall,F1
Average,0.748344,0.748563,0.748344,0.748322
Explicit,0.740260,0.547984,0.740260,0.629773
Implicit,0.756757,0.572681,0.756757,0.651975


In [47]:
predictions = []
metrics = trainer.evaluate(sub_test_dataset)
trainer.save_metrics('test', metrics)
save_preds(f'{save_dir}/test_predictions.json', predictions.tolist())

C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
test_df = pd.DataFrame(values.reshape(3, 4), columns=colnames, index=["Average", "Explicit", "Implicit"])

print("Test set metrics:")
test_df

Test set metrics:


,Accuracy,Precision,Recall,F1
Average,0.774834,0.775529,0.774834,0.774637
Explicit,0.762500,0.581406,0.762500,0.659752
Implicit,0.788732,0.622099,0.788732,0.695575


**Test using only correct Abusive predictions**

In [50]:
predictions = load_preds("./Models/hierarchical-initial/main")
df = load_csv("test", True)

# Create filtered sub model test dataset from predictions of main model
NA_indices = df[df["Class"] == 0].index.tolist()
A_indices = np.argwhere(predictions != 0).flatten()
A_indices = [x for x in A_indices if x not in NA_indices]

df = df.iloc[A_indices]
filtered_test_dataset = format_dataset(df)
filtered_test_dataset

Dataset({
    features: ['labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 135
})

In [52]:
predictions = []
metrics = trainer.evaluate(filtered_test_dataset)
trainer.save_metrics('test_filtered', metrics)
save_preds(f'{save_dir}/test_filtered_predictions.json', predictions.tolist())

colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
test_df = pd.DataFrame(values.reshape(3, 4), columns=colnames, index=["Average", "Explicit", "Implicit"])

print("Filtered test set metrics:")
test_df

Filtered test set metrics:


C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Accuracy,Precision,Recall,F1
Average,0.770370,0.770470,0.770370,0.769348
Explicit,0.769231,0.591716,0.769231,0.668896
Implicit,0.771930,0.595876,0.771930,0.672573


## Fine-tune pre-trained models

### Main model (NA v A)

In [45]:
save_dir = "./Models/hierarchical-tuning/main"

# Create directories
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [46]:
def objective(trial: optuna.Trial):
    config = RobertaConfig.from_pretrained(load_dir, num_labels=num_labels)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = RobertaAbusiveClassification.from_pretrained(load_dir, config=config)
    model.to(device)
    model.train()
    
    training_args = TrainingArguments(         
        output_dir= save_dir,
        optim = "adamw_torch",
        learning_rate = trial.suggest_float("learning_rate", 4e-5, 0.01, log=True),         
        weight_decay = trial.suggest_float("weight_decay", 4e-5, 0.01, log=True),         
        num_train_epochs = trial.suggest_int("num_train_epochs", low=2, high=5),         
        per_device_train_batch_size = 8,         
        per_device_eval_batch_size = 8,
        report_to = "none")

    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = main_train_dataset,
        eval_dataset = main_val_dataset)  
    
    result = trainer.train()     
    return result.training_loss

In [47]:
study = optuna.create_study(study_name="hp-search-robertaH-main", direction="minimize") 
study.optimize(func=objective, n_trials=128)

[I 2023-07-18 18:19:32,756] A new study created in memory with name: hp-search-robertaH-main
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAbusiveClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.

Step,Training Loss


[I 2023-07-18 18:20:28,425] Trial 0 finished with value: 0.3698821677301163 and parameters: {'learning_rate': 7.924610166825681e-05, 'weight_decay': 0.004973586884825482, 'num_train_epochs': 2}. Best is trial 0 with value: 0.3698821677301163.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of

Step,Training Loss
500,0.805700


[I 2023-07-18 18:22:22,617] Trial 1 finished with value: 0.7969553022456348 and parameters: {'learning_rate': 0.004005065781711942, 'weight_decay': 9.006508469742897e-05, 'num_train_epochs': 4}. Best is trial 0 with value: 0.3698821677301163.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of

...

[I 2023-07-18 23:03:52,615] Trial 125 finished with value: 0.18251371043069023 and parameters: {'learning_rate': 4.454445496012115e-05, 'weight_decay': 0.003175834749367224, 'num_train_epochs': 5}. Best is trial 53 with value: 0.18086845928565004.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weigh

Step,Training Loss
500,0.237300


[I 2023-07-18 23:06:17,858] Trial 126 finished with value: 0.18160812478316457 and parameters: {'learning_rate': 4.468676561693775e-05, 'weight_decay': 0.0026601842989247555, 'num_train_epochs': 5}. Best is trial 53 with value: 0.18086845928565004.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weig

Step,Training Loss


[I 2023-07-18 23:07:43,024] Trial 127 finished with value: 0.2907696296099135 and parameters: {'learning_rate': 4.012645327141113e-05, 'weight_decay': 0.002201506946379743, 'num_train_epochs': 3}. Best is trial 53 with value: 0.18086845928565004.


In [48]:
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])

print("Optimal hyperparamers:")
print(f'Learning rate\t: {best_lr}')
print(f'Weight decay\t: {best_weight_decay}')
print(f'Epochs\t\t: {best_epoch}')

Optimal hyperparamers:
Learning rate	: 4.817391461366549e-05
Weight decay	: 0.003826880119155053
Epochs		: 5


In [49]:
# Save best hyperparameters
data = {
    'learning_rate': best_lr,
    'weight_decay': best_weight_decay,
    'epoch': best_epoch
}

# Serializing json
json_object = json.dumps(data, indent=4)

# Writing to sample.json
with open(f'{save_dir}/best_hyperparameters.json', "w") as outfile:
    outfile.write(json_object)

### Sub model (E v I)

In [67]:
save_dir = "./Models/hierarchical-tuning/sub"

# Create directories
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [14]:
def objective(trial: optuna.Trial):
    config = RobertaConfig.from_pretrained(load_dir, num_labels=num_labels)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = RobertaAbusiveClassification.from_pretrained(load_dir, config=config)
    model.to(device)
    model.train()
    
    training_args = TrainingArguments(         
        output_dir= save_dir,
        optim = "adamw_torch",
        learning_rate = trial.suggest_float("learning_rate", 4e-5, 0.01, log=True),         
        weight_decay = trial.suggest_float("weight_decay", 4e-5, 0.01, log=True),         
        num_train_epochs = trial.suggest_int("num_train_epochs", low=2, high=5),         
        per_device_train_batch_size = 8,         
        per_device_eval_batch_size = 8,
        report_to = "none")

    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = sub_train_dataset,
        eval_dataset = sub_val_dataset)  
    
    result = trainer.train()     
    return result.training_loss

In [15]:
study = optuna.create_study(study_name="hp-search-robertaH-sub", direction="minimize") 
study.optimize(func=objective, n_trials=128)

[I 2023-07-20 04:10:45,256] A new study created in memory with name: hp-search-robertaH-sub
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAbusiveClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.o

Step,Training Loss


[I 2023-07-20 04:11:43,626] Trial 0 finished with value: 0.6394448713822798 and parameters: {'learning_rate': 0.00012179589962964528, 'weight_decay': 0.00015712083708170174, 'num_train_epochs': 3}. Best is trial 0 with value: 0.6394448713822798.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights

Step,Training Loss


[I 2023-07-20 04:12:17,036] Trial 1 finished with value: 0.8888439698652788 and parameters: {'learning_rate': 0.0014413291881208384, 'weight_decay': 0.0002625848732460458, 'num_train_epochs': 2}. Best is trial 0 with value: 0.6394448713822798.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights o

...

[I 2023-07-20 07:10:08,514] Trial 125 finished with value: 0.23073130520907315 and parameters: {'learning_rate': 6.043650233918616e-05, 'weight_decay': 0.000146147137394288, 'num_train_epochs': 5}. Best is trial 78 with value: 0.21921402324329722.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weigh

Step,Training Loss


[I 2023-07-20 07:11:40,160] Trial 126 finished with value: 0.22063302126797762 and parameters: {'learning_rate': 5.07962158420747e-05, 'weight_decay': 0.0001101862105202829, 'num_train_epochs': 5}. Best is trial 78 with value: 0.21921402324329722.
Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weigh

Step,Training Loss


[I 2023-07-20 07:12:35,346] Trial 127 finished with value: 0.36433893261533795 and parameters: {'learning_rate': 7.354600939060409e-05, 'weight_decay': 7.818822118884811e-05, 'num_train_epochs': 3}. Best is trial 78 with value: 0.21921402324329722.


In [16]:
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])

print("Optimal hyperparamers:")
print(f'Learning rate\t: {best_lr}')
print(f'Weight decay\t: {best_weight_decay}')
print(f'Epochs\t\t: {best_epoch}')

Optimal hyperparamers:
Learning rate	: 5.1987537900858175e-05
Weight decay	: 6.7077703057999e-05
Epochs		: 5


In [17]:
# Save best hyperparameters
data = {
    'learning_rate': best_lr,
    'weight_decay': best_weight_decay,
    'epoch': best_epoch
}

# Serializing json
json_object = json.dumps(data, indent=4)

# Writing to sample.json
with open(f'{save_dir}/best_hyperparameters.json', "w") as outfile:
    outfile.write(json_object)

## Training the best models

### Main model (NA v A)

In [18]:
# Variables
load_dir = "jcblaise/roberta-tagalog-base"
save_dir = "./Models/hierarchical-final/main"
hyperparameter_dir = "./Models/hierarchical-tuning/main"

# Create directories
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [19]:
# Load saved hyperparameters
with open(f'{hyperparameter_dir}/best_hyperparameters.json') as f:
    hyperparameters = json.load(f)

hyperparameters

{'learning_rate': 4.817391461366549e-05,
 'weight_decay': 0.003826880119155053,
 'epoch': 5}

In [20]:
config = RobertaConfig.from_pretrained(load_dir, num_labels=num_labels)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = RobertaAbusiveClassification.from_pretrained(load_dir, config=config)
model.to(device)
model.train()

training_args = TrainingArguments(
    output_dir= save_dir,
    learning_rate=hyperparameters['learning_rate'],
    weight_decay=hyperparameters['weight_decay'],
    num_train_epochs=hyperparameters['epoch'],
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    report_to = "none")

# Initialize Trainer class and train
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = main_train_dataset,
    eval_dataset = main_val_dataset,
    compute_metrics = compute_metrics_main)
result = trainer.train()

# Evaluate model and save metrics and predictions
model.eval()
predictions = []
metrics = trainer.evaluate()
trainer.save_metrics('validation', metrics)
save_preds(f'{save_dir}/validation_predictions.json', predictions.tolist())

# Save trained model
model.save_pretrained(f'{save_dir}/model')
tokenizer.save_pretrained(f'{save_dir}/model')

Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAbusiveClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.out_proj.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'classifier.out_p

Step,Training Loss
500,0.257200


C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('./Models/hierarchical-final/main/model\\tokenizer_config.json',
 './Models/hierarchical-final/main/model\\special_tokens_map.json',
 './Models/hierarchical-final/main/model\\vocab.json',
 './Models/hierarchical-final/main/model\\merges.txt',
 './Models/hierarchical-final/main/model\\added_tokens.json',
 './Models/hierarchical-final/main/model\\tokenizer.json')

In [21]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
val_df = pd.DataFrame(values.reshape(3, 4), columns=colnames, index=["Average", "Non-Abusive", "Abusive"])

print("Validation set metrics:")
val_df

Validation set metrics:


,Accuracy,Precision,Recall,F1
Average,0.818584,0.815152,0.818584,0.815419
Non-Abusive,0.757576,0.573921,0.757576,0.653083
Abusive,0.843750,0.711914,0.843750,0.772246


In [22]:
predictions = []
metrics = trainer.evaluate(main_test_dataset)
trainer.save_metrics('test', metrics)
save_preds(f'{save_dir}/test_predictions.json', predictions.tolist())

C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
test_df = pd.DataFrame(values.reshape(3, 4), columns=colnames, index=["Average", "Non-Abusive", "Abusive"])

print("Test set metrics:")
test_df

Test set metrics:


,Accuracy,Precision,Recall,F1
Average,0.828194,0.828780,0.828194,0.828470
Non-Abusive,0.740260,0.547984,0.740260,0.629773
Abusive,0.873333,0.762711,0.873333,0.814282


### Sub model (E v I)

In [53]:
# Variables
load_dir = "jcblaise/roberta-tagalog-base"
save_dir = "./Models/hierarchical-final/sub"
hyperparameter_dir = "./Models/hierarchical-tuning/sub"

# Create directories
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [54]:
# Load saved hyperparameters
#with open(f'{hyperparameter_dir}/best_hyperparameters.json') as f:

with open(f'{hyperparameter_dir}/best_hyperparameters.json') as f:
    hyperparameters = json.load(f)

hyperparameters

{'learning_rate': 5.1987537900858175e-05,
 'weight_decay': 6.7077703057999e-05,
 'epoch': 5}

In [80]:
# Attach classification head and prepare model for trainer
config = RobertaConfig.from_pretrained(load_dir, num_labels=num_labels)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_args = TrainingArguments(
    output_dir = save_dir,
    learning_rate=hyperparameters['learning_rate'],
    weight_decay=hyperparameters['weight_decay'],
    num_train_epochs=hyperparameters['epoch'],
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    report_to = "none")

# Initialize model
model = RobertaAbusiveClassification.from_pretrained(load_dir, config=config)
model.to(device)
model.train()

# Initialize Trainer class and train
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = sub_train_dataset,
    eval_dataset = sub_val_dataset,
    compute_metrics = compute_metrics_sub)
result = trainer.train()

# Evaluate model and save metrics and predictions
model.eval()
predictions = []
metrics = trainer.evaluate()
trainer.save_metrics('validation', metrics)
save_preds(f'{save_dir}/validation_predictions.json', predictions.tolist())

# Save trained model
model.save_pretrained(f'{save_dir}/model')
tokenizer.save_pretrained(f'{save_dir}/model')

Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaAbusiveClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAbusiveClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAbusiveClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.

Step,Training Loss


C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('./Models/hierarchical-final/sub/model\\tokenizer_config.json',
 './Models/hierarchical-final/sub/model\\special_tokens_map.json',
 './Models/hierarchical-final/sub/model\\vocab.json',
 './Models/hierarchical-final/sub/model\\merges.txt',
 './Models/hierarchical-final/sub/model\\added_tokens.json',
 './Models/hierarchical-final/sub/model\\tokenizer.json')

In [82]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
val_df = pd.DataFrame(values.reshape(3, 4), columns=colnames, index=["Average", "Explicit", "Implicit"])

print("Validation set metrics:")
val_df

Validation set metrics:


,Accuracy,Precision,Recall,F1
Average,0.781457,0.785683,0.781457,0.780765
Explicit,0.750000,0.562500,0.750000,0.642857
Implicit,0.820896,0.673869,0.820896,0.740152


In [56]:
predictions = []
metrics = trainer.evaluate(sub_test_dataset)
trainer.save_metrics('test', metrics)
save_preds(f'{save_dir}/test_predictions.json', predictions.tolist())

C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
test_df = pd.DataFrame(values.reshape(3, 4), columns=colnames, index=["Average", "Explicit", "Implicit"])

print("Test set metrics:")
test_df

Test set metrics:


,Accuracy,Precision,Recall,F1
Average,0.768212,0.771899,0.768212,0.767293
Explicit,0.741176,0.549343,0.741176,0.631002
Implicit,0.803030,0.644858,0.803030,0.715304


**Test using only correct Abusive predictions**

In [59]:
predictions = load_preds("./Models/hierarchical-final/main")
df = load_csv("test", True)

# Create filtered sub model test dataset from predictions of main model
NA_indices = df[df["Class"] == 0].index.tolist()
A_indices = np.argwhere(predictions != 0).flatten()
A_indices = [x for x in A_indices if x not in NA_indices]

df = df.iloc[A_indices]
filtered_test_dataset = format_dataset(df)
filtered_test_dataset

Dataset({
    features: ['labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 131
})

In [60]:
predictions = []
metrics = trainer.evaluate(filtered_test_dataset)
trainer.save_metrics('test_filtered', metrics)
save_preds(f'{save_dir}/test_filtered_predictions.json', predictions.tolist())

colnames = ["Accuracy", "Precision", "Recall", "F1"]
values = np.array(list(metrics.values())[1:-4])
test_df = pd.DataFrame(values.reshape(3, 4), columns=colnames, index=["Average", "Explicit", "Implicit"])

print("Filtered test set metrics:")
test_df

Filtered test set metrics:


C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jan\.conda\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Accuracy,Precision,Recall,F1
Average,0.755725,0.757545,0.755725,0.752032
Explicit,0.746988,0.557991,0.746988,0.638803
Implicit,0.770833,0.594184,0.770833,0.671078
